In [1]:
import statsapi
import operator
import json

In [2]:
MLB_teams = ["Angels", "Astros", "Athletics", "Blue Jays", "Braves", "Brewers", "Cardinals", "Cubs", "Diamondbacks", "Dodgers", "Giants", "Guardians", "Mariners", "Marlins", "Mets", "Nationals", "Orioles", "Padres", "Phillies", "Pirates", "Rangers", "Rays", "Red Sox", "Reds", "Rockies", "Royals", "Tigers", "Twins", "White Sox", "Yankees"]


In [3]:
def get_team_ID(team_name):
    teams = statsapi.lookup_team(team_name)
    for team in teams:
        if team_name.lower() in team['name'].lower():
            return team['id']
        else:
            print('Team not found')
            return 0
    

In [4]:
def get_season_schedule(team, year, month=0, day=0):
    team_ID = get_team_ID(team)
    if day ==0 and month ==0:
        schedule = statsapi.schedule(start_date=f"01/01/{year}",end_date=f"12/31/{year}",team= team_ID)
    elif day == 0:
        schedule = statsapi.schedule(start_date=f"{month}/01/{year}",end_date=f"{month}/31/{year}",team= team_ID)
    elif month ==0:
        schedule = statsapi.schedule(start_date=f"01/{day}/{year}",end_date=f"12/{day}/{year}",team= team_ID)
    else: 
        schedule = statsapi.schedule(start_date=f"{month}/{day}/{year}",end_date=f"{month}/{day}/{year}",team= team_ID)
    count = 1
    team_schedule = []
    for game in schedule:
        if game['game_type'] == 'R' and game['away_score'] != game['home_score']:
            team_schedule.append(game)
            count+=1
    return team_schedule
# STL_2011 = get_season_schedule('Cardinals',2011,8)
# for game in STL_2011:
#     print(game['game_id'])

In [5]:
# Get a full season's game IDs as a list
def season_game_IDs(year):
    game_IDs= []
    for team in MLB_teams:
        team_ID = get_team_ID(team)
        schedule = statsapi.schedule(start_date=f"01/01/{year}",end_date=f"12/31/{year}",team= team_ID)
        team_schedule = []
        for game in schedule:
            if game['game_type'] == 'R' and game['away_score'] != game['home_score']:
                team_schedule.append(game['game_id'])
        for id in team_schedule:
            if id not in game_IDs:
                game_IDs.append(id)
    return game_IDs

game_IDs_2023 = season_game_IDs(2023)
    

In [7]:
print(game_IDs_2023)

[718769, 718757, 718734, 718720, 718706, 718691, 718670, 718657, 718644, 718631, 718614, 718602, 718580, 718576, 718559, 718540, 718529, 718504, 718496, 718485, 718466, 718452, 718437, 718425, 718408, 718402, 718386, 718376, 718365, 718338, 718320, 718311, 718294, 718275, 718264, 718254, 718237, 718235, 718205, 718191, 718182, 718171, 718154, 718137, 718128, 718106, 718096, 718080, 718070, 718053, 718034, 718015, 717998, 717984, 717971, 717959, 717943, 717936, 717926, 717908, 717899, 717861, 717855, 717838, 717824, 717809, 717799, 717792, 717780, 717765, 717751, 717736, 717726, 717712, 717678, 717662, 717644, 717640, 717616, 717608, 717592, 717576, 717560, 717549, 717533, 717522, 717505, 717497, 717478, 717456, 717437, 717405, 717389, 717381, 717369, 717352, 717339, 717312, 717297, 717278, 717265, 717243, 717247, 717235, 717222, 717201, 717187, 717176, 717165, 717144, 717130, 717109, 717101, 717090, 717068, 717071, 717037, 717022, 717004, 716997, 716985, 716971, 716945, 716915, 716925,

In [42]:
# Write a single highlight to a csv file
import csv
def highlight_to_csv(highlight_str,year):
    csv_file_path = f"../data/highlights_{year}.csv"

    with open(csv_file_path, 'a', newline='',encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(highlight_str)

# sample game_id = 718437
# write all the highlights from one game to a csv
def game_highlights_to_csv(game_id):
    data = statsapi.game_highlight_data(game_id)
    highlights = statsapi.game_highlight_data(game_id)
    file_path = f"../data/highlights_{game_id}.json"

    # Write data to the CSV file
    for highlight in highlights:
        headline = highlight["headline"]
        date = highlight["date"]
        year = int(date[:4])
        if 'blurb' not in highlight and 'description' not in highlight:
            blurb = "no blurb available"
            description = "no description available"
        else:
            if 'blurb' in highlight:
                blurb = highlight["blurb"]
            else: blurb = highlight["description"]    
            if 'description' in highlight:
                description = highlight["description"]
            else: description = blurb
        mp4_url = highlight["playbacks"][0]["url"]
        duration = highlight["duration"]
        
        has_player = any(item['type'] == 'player_id' for item in highlight['keywordsAll'])
        if has_player:
            player_info = [(item['value'], item['displayName']) for item in highlight['keywordsAll'] if item['type'] == 'player_id']
        else:
            player_info = [(0, 'NA')]
        player_id = player_info[0][0]
        player_name = player_info[0][1]


        highlight_list = [game_id,date,player_name,player_id,headline,blurb,description,mp4_url,duration]
    
        highlight_to_csv(highlight_list,year)


In [6]:

import json

game_id = 718437
# write all the highlights from one game to a json
def game_highlights_to_json(game_id):
    data = statsapi.game_highlight_data(game_id)
    highlights = statsapi.game_highlight_data(game_id)
    file_path = f"../data/highlights_{game_id}.json"


    game_highlights = []
    for highlight in highlights:
        headline = highlight["headline"]
        date = highlight["date"]
        year = int(date[:4])
        if 'blurb' not in highlight and 'description' not in highlight:
            blurb = "no blurb available"
            description = "no description available"
        else:
            if 'blurb' in highlight:
                blurb = highlight["blurb"]
            else: blurb = highlight["description"]    
            if 'description' in highlight:
                description = highlight["description"]
            else: description = blurb
        mp4_url = highlight["playbacks"][0]["url"]
        
        has_player = any(item['type'] == 'player_id' for item in highlight['keywordsAll'])
        if has_player:
            player_info = [(item['value'], item['displayName']) for item in highlight['keywordsAll'] if item['type'] == 'player_id']
        else:
            player_info = [(0, 'NA')]
        player_id = player_info[0][0]
        player_name = player_info[0][1]

        #FIND API CALL FOR GAME DATA
        boxscore = statsapi.boxscore_data(game_id)
        away_name = boxscore["teamInfo"]["away"]["teamName"]
        away_id = boxscore["teamInfo"]["away"]["id"]
        home_name = boxscore["teamInfo"]["home"]["teamName"]
        home_id = boxscore["teamInfo"]["home"]["id"]
        highlight_dict = {"game_id":game_id,"date":date,"away_name":away_name,"away_id":away_id,"home_name":home_name,"home_id":home_id,"player_name":player_name,"player_id":player_id,"headline":headline,"blurb":blurb,"description":description,"mp4_url":mp4_url}
        game_highlights.append(highlight_dict)

            # Write data to the JSON file
    with open(file_path, "w") as json_file:
        json.dump(game_highlights, json_file)



In [46]:
for i in range(0,2430):
    game_id = game_IDs_2023[i]
    game_highlights_to_csv(game_id)

In [26]:

import csv

# sample game_id = 718437
# write game data from one game to a csv
def game_data_to_csv(game_id):
    boxscore = statsapi.boxscore_data(game_id)
    date = boxscore["gameId"][:10]
    year = int(date[:4])

    away_name = boxscore["teamInfo"]["away"]["teamName"]
    away_id = boxscore["teamInfo"]["away"]["id"]
    home_name = boxscore["teamInfo"]["home"]["teamName"]
    home_id = boxscore["teamInfo"]["home"]["id"]
    away_score = boxscore["away"]["teamStats"]["batting"]["runs"]
    home_score= boxscore["home"]["teamStats"]["batting"]["runs"]

    game_data = [game_id,date,away_name,away_id,away_score,home_name,home_id,home_score]

    csv_file_path = f"../data/game_data_{year}.csv"

    with open(csv_file_path, 'a', newline='',encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(game_data)


In [40]:

for i in range(0,2430):
    game_data_to_csv(game_IDs_2023[i])


KeyboardInterrupt: 

In [35]:
import statsapi
from datetime import datetime, timedelta
def yesterday(frmt='%Y-%m-%d', string=True):
    yesterday = datetime.now() - timedelta(1)
    if string:
        return yesterday.strftime(frmt)
    return yesterday

y_schedule = statsapi.schedule(yesterday())
y_game_ids = []
for game in y_schedule:
    y_game_ids.append(game['game_id'])
print(y_game_ids)


for game in y_game_ids:
    game_highlights_to_csv(game)

[748093, 748089, 748092, 748090, 748083, 748085, 748080, 748084, 748087, 748082, 748086, 748088, 748091]


In [37]:
import json

highlights statsapi.game_highlight_data(748093)

[{'type': 'video', 'state': 'A', 'date': '2024-03-06T21:56:13.317Z', 'id': 'wilyer-abreu-hits-a-solo-home-run-to-right-center', 'headline': 'Wilyer Abreu hits a solo home run to right-center', 'seoTitle': '', 'slug': 'wilyer-abreu-hits-a-solo-home-run-to-right-center', 'blurb': 'Wilyer Abreu hits a solo home run to right-center', 'keywordsAll': [{'type': 'game', 'value': 'gamepk-748093', 'displayName': '2024/03/06 BOS@MIN'}, {'type': 'game_pk', 'value': '748093', 'displayName': '2024/03/06 BOS@MIN'}, {'type': 'team', 'value': 'teamid-111', 'displayName': 'Boston Red Sox'}, {'type': 'team_id', 'value': '111', 'displayName': 'Boston Red Sox'}, {'type': 'player', 'value': 'playerid-677800', 'displayName': 'Wilyer Abreu'}, {'type': 'player_id', 'value': '677800', 'displayName': 'Wilyer Abreu'}, {'type': 'taxonomy', 'value': 'vod', 'displayName': 'vod'}, {'type': 'taxonomy', 'value': 'spring-training', 'displayName': 'Spring Training'}, {'type': 'taxonomy', 'value': 'highlight', 'displayNam